In [32]:
import pandas as pd
import requests
import os, json
from json import JSONDecodeError

In [6]:
def get_json(from_year, to_year):
    if not 2000 < int(from_year) < 2050 or not 2000 < int(from_year) < 2050:
        return []
    all_holidays = []
    for i in range(from_year, to_year+1):
        r = requests.get(f'https://date.nager.at/api/v2/publicholidays/{i}/CH')
        all_holidays.extend(r.json())
    return all_holidays

def set_date_index(df, col='date'):
    return df.set_index(pd.to_datetime(df[col])).drop(col, axis=1)

def filter_canton(df, can='BS'):
    return df[[(str('CH-' + can) in row) if row is not None else True for row in df.counties]]

def get_holiday_data(canton='BS', from_year=2018, to_year=2018):
    holiday_data = get_json(from_year, to_year)
    holiday_df = (pd.DataFrame.from_records(holiday_data)
                  .pipe(set_date_index)
                  .pipe(filter_canton, can=holiday_canton))
    return holiday_df

In [33]:
# 105: Basel
def get_weather_data(station_index=105):    
    stationdata = []
    station_error = 0

    for root, dirs, files in os.walk("weather_data/2018"):
        for file in files: 
            if not file.endswith(".json"): continue
            filename = os.path.join(root, file)
            try:
                with open(filename) as json_file:
                    data = json.load(json_file)
                    stationdata.append(data[station_index])
            except UnicodeDecodeError:
                station_error = station_error + 1
            except JSONDecodeError:
                station_error = station_error + 1
    print ("Loaded: %d - errors: %d" % (len(stationdata), station_error))
    pandata = pd.DataFrame(stationdata)
    pandata = pandata.drop(columns=['code','station'])
    return pandata
get_weather_data()

Loaded: 13690 - errors: 554


,dateTime,gustPeak,humidity,precipitation,qfePressure,qffPressure,qnhPressure,sunshine,temperature,windDirection,windSpeed
0,2018-11-09T17:40:00.000Z,2.9,79,0.0,976.6,1014.4,1014.2,0,10.2,67,1.1
1,2018-03-13T14:30:00.000Z,37.8,69,0.0,972.8,1010.8,1010.3,0,8.3,252,20.5
2,2018-03-22T00:00:00.000Z,3.2,79,0.0,987.7,1027.9,1025.6,0,-2.6,193,2.2
3,2018-04-05T03:40:00.000Z,15.1,74,0.0,973.8,1011.6,1011.3,0,9.9,270,4.0
4,2018-04-29T23:50:00.000Z,None,None,None,None,None,None,None,None,None,None
5,2018-11-26T03:10:00.000Z,10.4,84,0.0,967.6,1005.9,1004.9,0,4.2,100,4.0
6,2018-11-10T11:30:00.000Z,26.3,68,0.0,971.8,1009.0,1009.3,8,13.1,75,13.3
7,2018-12-09T20:20:00.000Z,15.1,87,0.0,979.1,1017.6,1016.8,0,6.3,263,8.3
8,2018-04-05T12:50:00.000Z,33.5,83,0.0,980.0,1018.0,1017.7,0,10.0,273,18.0
9,2018-03-08T21:10:00.000Z,3.2,47,0.0,970.8,1008.5,1008.2,0,9.4,135,1.4
